<a href="https://colab.research.google.com/github/zain-codes/HomeAway-Insights/blob/main/Hypo1_ML_model_forecasting_profitability_for_potential_real_estate_investments_in_towns_with_varying_levels_of_tourist_activity_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import auth


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
!pip install openpyxl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Table 1**

In [ ]:
#POI_tourist_establishments = """select * from Prello.POI_tourist_establishments"""
#POI_tourist_establishments = client.query(POI_tourist_establishments).result().to_dataframe()

# To read the first sheet or a specific sheet
POI_tourist_establishments = pd.read_excel('/content/drive/MyDrive/Lewagon Prello full dataset/POI_tourist_establishments.xlsx', engine='openpyxl')
POI_tourist_establishments


,poi,name,latitude,longitude,municipality_code,importance,name_reprocessed
0,hotel,Club Med (Hôtel),45.678531,-1.163635,17225,0.078556,Club Med
1,hotel,Cabanes des Grands Cépages (Hôtel),44.026859,4.847491,84129,0.078419,Cabanes des Grands Cépages
2,hotel,Pierre et Vacances (Hôtel),50.331541,1.565677,80333,0.077999,Pierre et Vacances
3,hotel,Pierre & Vacances (Hôtel),43.285936,6.569696,83068,0.077702,Pierre & Vacances
4,hotel,Le Domaine du Golf (Hôtel),43.541050,3.752852,34095,0.077542,Le Domaine du Golf
...,...,...,...,...,...,...,...
26211,camp_site,Camping municipal,45.701000,2.041247,19164,0.040000,Camping municipal
26212,camp_site,Camping municipal,46.491559,3.020672,03238,0.040000,Camping municipal
26213,camp_site,Camping municipal,45.694389,2.120544,19241,0.040000,Camping municipal
26214,camp_site,Camping municipal,45.915488,2.487122,23131,0.040000,Camping municipal


In [ ]:
POI_tourist_establishments

,poi,name,latitude,longitude,municipality_code,importance,name_reprocessed
0,hotel,Club Med (Hôtel),45.678531,-1.163635,17225,0.078556,Club Med
1,hotel,Cabanes des Grands Cépages (Hôtel),44.026859,4.847491,84129,0.078419,Cabanes des Grands Cépages
2,hotel,Pierre et Vacances (Hôtel),50.331541,1.565677,80333,0.077999,Pierre et Vacances
3,hotel,Pierre & Vacances (Hôtel),43.285936,6.569696,83068,0.077702,Pierre & Vacances
4,hotel,Le Domaine du Golf (Hôtel),43.541050,3.752852,34095,0.077542,Le Domaine du Golf
...,...,...,...,...,...,...,...
26211,camp_site,Camping municipal,45.701000,2.041247,19164,0.040000,Camping municipal
26212,camp_site,Camping municipal,46.491559,3.020672,03238,0.040000,Camping municipal
26213,camp_site,Camping municipal,45.694389,2.120544,19241,0.040000,Camping municipal
26214,camp_site,Camping municipal,45.915488,2.487122,23131,0.040000,Camping municipal


In [ ]:
POI_tourist_establishments.columns

Index(['poi', 'name', 'latitude', 'longitude', 'municipality_code',
       'importance', 'name_reprocessed'],
      dtype='object')

In [ ]:
POI_tourist_establishments.isnull().sum()

poi                  0
name                 0
latitude             0
longitude            0
municipality_code    0
importance           0
name_reprocessed     0
dtype: int64

In [ ]:
POI_tourist_establishments.columns

Index(['poi', 'name', 'latitude', 'longitude', 'municipality_code',
       'importance', 'name_reprocessed'],
      dtype='object')

In [ ]:
POI_tourist_establishments.drop_duplicates(subset= ["poi", "municipality_code"], inplace= True)

In [ ]:
POI_tourist_establishments["poi"].unique()

array(['hotel', 'hostel', 'camp_site', 'guest_house', 'bed_and_breakfast'],
      dtype=object)

In [ ]:
POI_tourist_establishments= POI_tourist_establishments.drop_duplicates(subset=['municipality_code'], keep='first')
POI_tourist_establishments["municipality_code"].duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
26210    False
26211    False
26212    False
26214    False
26215    False
Name: municipality_code, Length: 8828, dtype: bool

**Table 2**

In [ ]:
#POI_touristic_sites_by_municipality = """select * from nth-anchor-414009.Prello.POI_touristic_sites_by_municipality"""
#POI_touristic_sites_by_municipality  = client.query(POI_touristic_sites_by_municipality).result().to_dataframe()

# To read the first sheet or a specific sheet
POI_touristic_sites_by_municipality= pd.read_excel('/content/drive/MyDrive/Lewagon Prello full dataset/POI_touristic_sites_by_municipality.xlsx', engine='openpyxl')
POI_touristic_sites_by_municipality

,poi,name,latitude,longitude,municipality_code,importance,name_reprocessed
0,1,Fortifications de Vauban (Patrimoine mondial),46.396160,1.472600,36134,0.139527,Fortifications de Vauban
1,1,Chemins de Saint-Jacques-de-Compostelle en Fra...,46.313695,1.478772,23258,0.137821,Chemins de Saint-Jacques-de-Compostelle en Fra...
2,1,Canal du Midi (Patrimoine mondial),43.332709,3.219890,34032,0.129531,Canal du Midi
3,1,Bassin minier du Nord-Pas de Calais (Patrimoin...,50.455895,2.965034,62427,0.127170,Bassin minier du Nord-Pas de Calais
4,1,"Les Causses et les Cévennes, paysage culturel ...",44.346210,3.613406,48050,0.124981,"Les Causses et les Cévennes, paysage culturel ..."
...,...,...,...,...,...,...,...
31029,viewpoint,Tour Eiffel 3e étage (Point de vue),48.858262,2.294497,75056,0.053782,Tour Eiffel 3e étage
31030,theme_park,Jardin d'Acclimatation (Parc à thème),48.878145,2.264686,75056,0.087097,Jardin d'Acclimatation
31031,theme_park,Foire du Trône (Parc à thème),48.832003,2.404337,75056,0.060000,Foire du Trône
31032,golf_course,Golf du Bois de Boulogne (Terrain de golf),48.857776,2.232651,75056,0.060073,Golf du Bois de Boulogne


In [ ]:
POI_touristic_sites_by_municipality .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31034 entries, 0 to 31033
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   poi                31034 non-null  object 
 1   name               31034 non-null  object 
 2   latitude           31034 non-null  float64
 3   longitude          31034 non-null  float64
 4   municipality_code  31034 non-null  object 
 5   importance         31034 non-null  float64
 6   name_reprocessed   31034 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.7+ MB


In [ ]:
POI_touristic_sites_by_municipality.isnull().sum()

poi                  0
name                 0
latitude             0
longitude            0
municipality_code    0
importance           0
name_reprocessed     0
dtype: int64

In [ ]:
POI_touristic_sites_by_municipality.columns

Index(['poi', 'name', 'latitude', 'longitude', 'municipality_code',
       'importance', 'name_reprocessed'],
      dtype='object')

In [ ]:
POI_touristic_sites_by_municipality ["municipality_code"].duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
31029     True
31030     True
31031     True
31032     True
31033     True
Name: municipality_code, Length: 31034, dtype: bool

In [ ]:
POI_touristic_sites_by_municipality .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31034 entries, 0 to 31033
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   poi                31034 non-null  object 
 1   name               31034 non-null  object 
 2   latitude           31034 non-null  float64
 3   longitude          31034 non-null  float64
 4   municipality_code  31034 non-null  object 
 5   importance         31034 non-null  float64
 6   name_reprocessed   31034 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.7+ MB


**Table 3**

In [ ]:
#housing_stock = """select * from nth-anchor-414009.Prello.housing_stock"""
#housing_stock = client.query(housing_stock).result().to_dataframe()

# To read the first sheet or a specific sheet
housing_stock = pd.read_excel('/content/drive/MyDrive/Lewagon Prello full dataset/housing_stock.xlsx', engine='openpyxl')
housing_stock

,int64_field_0,municipality_code,year,nb_principal_home,nb_second_home,nb_vacants_housing,nb_tot_housing,secondary_home_rate,principal_home_rate,vacants_housing_rate,country_code
0,25,01339,1968,109,155,0,264,0.587121,0.412879,0.000000,1
1,89,02368,1968,132,56,0,188,0.297872,0.702128,0.000000,1
2,203,04073,1968,134,93,0,227,0.409692,0.590308,0.000000,1
3,238,04148,1968,42,93,0,135,0.688889,0.311111,0.000000,1
4,270,05012,1968,70,53,0,123,0.430894,0.569106,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
279579,275947,13105,2018,2951,48,202,3201,0.014995,0.921899,0.063105,1
279580,276174,78380,2018,2325,48,205,2578,0.018619,0.901862,0.079519,1
279581,276753,95352,2018,1918,48,212,2178,0.022039,0.880624,0.097337,1
279582,277014,59279,2018,8137,48,471,8656,0.005545,0.940042,0.054413,1


In [ ]:
housing_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279584 entries, 0 to 279583
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   int64_field_0         279584 non-null  int64  
 1   municipality_code     279584 non-null  object 
 2   year                  279584 non-null  int64  
 3   nb_principal_home     279584 non-null  int64  
 4   nb_second_home        279584 non-null  int64  
 5   nb_vacants_housing    279584 non-null  int64  
 6   nb_tot_housing        279584 non-null  int64  
 7   secondary_home_rate   279584 non-null  float64
 8   principal_home_rate   279584 non-null  float64
 9   vacants_housing_rate  279584 non-null  float64
 10  country_code          279584 non-null  int64  
dtypes: float64(3), int64(7), object(1)
memory usage: 23.5+ MB


In [ ]:
housing_stock.isnull().sum()

int64_field_0           0
municipality_code       0
year                    0
nb_principal_home       0
nb_second_home          0
nb_vacants_housing      0
nb_tot_housing          0
secondary_home_rate     0
principal_home_rate     0
vacants_housing_rate    0
country_code            0
dtype: int64

In [ ]:
housing_stock["municipality_code"].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
279579     True
279580     True
279581     True
279582     True
279583     True
Name: municipality_code, Length: 279584, dtype: bool

**Table 4** Financial info 1

In [ ]:
#average_salary_by_municipality = """select * from nth-anchor-414009.Prello.average_salary_by_municipality"""
#average_salary_by_municipality = client.query(average_salary_by_municipality).result().to_dataframe()

# To read the first sheet or a specific sheet
average_salary_by_municipality = pd.read_excel('/content/drive/MyDrive/Lewagon Prello full dataset/average_salary_by_municipality.xlsx', engine='openpyxl')
average_salary_by_municipality

,municipality_code,avg_net_salary,year,country_code
0,01004,26471.026727,2019,1
1,01007,26341.353419,2019,1
2,01014,25897.497842,2019,1
3,01024,25695.240341,2019,1
4,01025,26054.712323,2019,1
...,...,...,...,...
26670,97420,23364.247855,2015,1
26671,97421,19099.408481,2015,1
26672,97422,21854.551578,2015,1
26673,97423,20863.807536,2015,1


**Table 5** Financial info 2

In [ ]:
#real_estate_info_by_municipality = """select * from nth-anchor-414009.Prello.real_estate_info_by_municipality"""
#real_estate_info_by_municipality = client.query(real_estate_info_by_municipality).result().to_dataframe()

# To read the first sheet or a specific sheet
real_estate_info_by_municipality = pd.read_excel('/content/drive/MyDrive/Lewagon Prello full dataset/real_estate_info_by_municipality.xlsx', engine='openpyxl')
real_estate_info_by_municipality

,municipality_code,intensite_tension_immo,rental_max_apartment,rental_min_apartment,rental_med_house,rental_max_house,rental_min_house,rental_med_all,rental_max_all,rental_min_all
0,57133,8,12.27,9.07,9.19,14.45,6.64,9.53,13.77,7.25
1,57446,8,18.22,7.69,10.92,14.16,7.77,11.09,15.97,7.73
2,77013,9,18.30,8.39,12.20,16.71,9.21,12.26,16.75,9.19
3,77026,9,15.76,6.28,9.51,13.20,7.04,9.53,13.23,7.03
4,77072,9,17.50,7.69,11.47,15.75,8.21,11.47,15.75,8.21
...,...,...,...,...,...,...,...,...,...,...
34436,81126,7,16.64,6.36,8.93,11.28,6.45,8.93,11.28,6.45
34437,33425,11,16.20,7.90,11.07,14.72,7.40,11.09,14.74,7.41
34438,85146,5,18.51,8.22,10.59,13.50,8.43,10.88,15.47,8.35
34439,53062,8,16.12,6.28,8.32,11.28,6.15,8.69,12.93,6.19


**Table 7**

In [ ]:
#population_by_municipality = """select * from nth-anchor-414009.Prello.population_by_municipality"""
#population_by_municipality = client.query(population_by_municipality).result().to_dataframe()

# To read the first sheet or a specific sheet
population_by_municipality = pd.read_excel('/content/drive/MyDrive/Lewagon Prello full dataset/population_by_municipality.xlsx', engine='openpyxl')
population_by_municipality

,municipality_code,year,population,country_code
0,04213,1968,0.0,1
1,05137,1968,0.0,1
2,55039,1968,0.0,1
3,55050,1968,0.0,1
4,55239,1968,0.0,1
...,...,...,...,...
689490,67482,2019,287228.0,1
689491,93051,2019,67871.0,1
689492,92012,2019,121583.0,1
689493,33063,2019,260958.0,1


In [ ]:
population_by_municipality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689495 entries, 0 to 689494
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   municipality_code  689495 non-null  object 
 1   year               689495 non-null  int64  
 2   population         689495 non-null  float64
 3   country_code       689495 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 21.0+ MB


In [ ]:
population_by_municipality.isnull().sum()

municipality_code    0
year                 0
population           0
country_code         0
dtype: int64

In [ ]:
population_by_municipality.isnull().sum()

municipality_code    0
year                 0
population           0
country_code         0
dtype: int64

In [ ]:
population_duplicates =population_by_municipality["municipality_code"].duplicated()
population_duplicates

0         False
1         False
2         False
3         False
4         False
          ...  
689490     True
689491     True
689492     True
689493     True
689494     True
Name: municipality_code, Length: 689495, dtype: bool

###Remove Unnecessary Columns:

In [ ]:
# Column that is just an index with no analytical value
housing_stock.drop(['int64_field_0'], axis=1, inplace=True) # if you face an error it means that the column is already dropped

###Handle Missing Values

In [ ]:
# For numeric columns, fill missing values with the mean
population_by_municipality['population'].fillna(population_by_municipality['population'].mean(), inplace=True)

POI_tourist_establishments['importance'].fillna(POI_tourist_establishments['importance'].mean(), inplace=True)

# If there's a column with categorical data, fill missing values with the mode or a placeholder value
POI_tourist_establishments['name'].fillna('Unknown', inplace=True)

POI_touristic_sites_by_municipality['importance'].fillna(POI_touristic_sites_by_municipality['importance'].mean(), inplace=True)


<ipython-input-30-b0c2402607e9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POI_tourist_establishments['importance'].fillna(POI_tourist_establishments['importance'].mean(), inplace=True)
<ipython-input-30-b0c2402607e9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POI_tourist_establishments['name'].fillna('Unknown', inplace=True)


###Correct Data Types

In [ ]:
# municipality_code should be an integer or a string if it's categorical and not used for calculations
population_by_municipality['municipality_code'] = population_by_municipality['municipality_code'].astype(str)

POI_tourist_establishments['municipality_code'] = POI_tourist_establishments['municipality_code'].astype(str)

<ipython-input-31-95e3340a5693>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POI_tourist_establishments['municipality_code'] = POI_tourist_establishments['municipality_code'].astype(str)


### Remove duplicate rows to prevent skewed analysis

In [ ]:
POI_tourist_establishments.drop_duplicates(subset=['municipality_code'], inplace=True)
POI_touristic_sites_by_municipality.drop_duplicates(subset=['municipality_code'], inplace=True)

<ipython-input-32-47b6b0a76f42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POI_tourist_establishments.drop_duplicates(subset=['municipality_code'], inplace=True)


###Normalize Data

In [ ]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

housing_stock[['nb_principal_home', 'nb_second_home']] = scaler.fit_transform(housing_stock[['nb_principal_home', 'nb_second_home']])

###Create Consistent Column Names

In [ ]:
# Renaming 'name_reprocessed' column in both dataframes
POI_tourist_establishments.rename(columns={'name_reprocessed': 'establishment_name'}, inplace=True)
POI_touristic_sites_by_municipality.rename(columns={'name_reprocessed': 'site_name'}, inplace=True)


<ipython-input-34-1ceaa93aa67a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  POI_tourist_establishments.rename(columns={'name_reprocessed': 'establishment_name'}, inplace=True)


###Verify Integrity

In [ ]:
# Check for any rows where the sum does not equal the total housing
housing_check = (housing_stock['nb_principal_home'] + housing_stock['nb_second_home'] + housing_stock['nb_vacants_housing']) != housing_stock['nb_tot_housing']
# This will give a boolean series; you can decide how to handle rows that don't match
housing_check

0         True
1         True
2         True
3         True
4         True
          ... 
279579    True
279580    True
279581    True
279582    True
279583    True
Length: 279584, dtype: bool

### Merge Datasets

In [ ]:
import pandas as pd

# Merge POI tourist establishments and sites with an indicator
poi_combined = pd.merge(
    POI_tourist_establishments,
    POI_touristic_sites_by_municipality,
    on='municipality_code',
    how='outer',
    suffixes=('_establishment', '_site'),
    indicator=True
)

# Check the merge results by examining the '_merge' column
print(poi_combined['_merge'].value_counts())

# Merge the combined POI data with housing stock with an indicator
poi_housing_combined = pd.merge(
    poi_combined,
    housing_stock,
    on='municipality_code',
    how='outer',
    indicator='housing_stock_merge'
)

# Check the merge results by examining the 'housing_stock_merge' column
print(poi_housing_combined['housing_stock_merge'].value_counts())

# Merge the combined POI and housing data with the population data with an indicator
final_combined_data = pd.merge(
    poi_housing_combined,
    population_by_municipality,
    on='municipality_code',
    how='outer',
    indicator='population_merge'
)

# Check the merge results by examining the 'population_merge' column
print(final_combined_data['population_merge'].value_counts())

# The final_combined_data DataFrame now has an additional column for each merge indicator.
# You may want to drop these indicator columns if you no longer need them.
final_combined_data.drop(columns=['_merge', 'housing_stock_merge', 'population_merge'], inplace=True)

# Optional: Save the combined data to a CSV file for further analysis
final_combined_data.to_csv('combined_data.csv', index=False)

# Print the head of the final combined dataframe to check the first few rows.
print(final_combined_data.head())

_merge
right_only    6650
both          5369
left_only     3459
Name: count, dtype: int64
housing_stock_merge
right_only    155760
both          123824
left_only          0
Name: count, dtype: int64
population_merge
both          5304776
right_only      26398
left_only           8
Name: count, dtype: int64
  poi_establishment name_establishment  latitude_establishment  \
0             hotel   Club Med (Hôtel)               45.678531   
1             hotel   Club Med (Hôtel)               45.678531   
2             hotel   Club Med (Hôtel)               45.678531   
3             hotel   Club Med (Hôtel)               45.678531   
4             hotel   Club Med (Hôtel)               45.678531   

   longitude_establishment municipality_code  importance_establishment  \
0                -1.163635             17225                  0.078556   
1                -1.163635             17225                  0.078556   
2                -1.163635             17225                  0.078556   

### Post-Merge Data Cleaning

In [ ]:
# Check for any inconsistencies post-merge
# Example: Ensure that population figures are non-negative
final_combined_data['population'] = final_combined_data['population'].clip(lower=0)

# Handle any missing values that may have been introduced by the merge
# Example: Fill missing population with the median value
final_combined_data['population'].fillna(final_combined_data['population'].median(), inplace=True)

# Verify merges - If you've already printed value_counts after each merge, you might skip this step
# However, if the '_merge' columns have been dropped, and you wish to do additional checks, you might consider checking unique values, etc.
# Example: Ensure that all municipalities in the final data have corresponding population data
assert final_combined_data['population'].notnull().all()

In [ ]:
# First, aggregate the number of tourist attractions by municipality_code
attraction_counts = POI_tourist_establishments.groupby('municipality_code').size().reset_index(name='total_tourist_attractions')

# Merge this with your final_combined_data DataFrame
final_combined_data = pd.merge(final_combined_data, attraction_counts, on='municipality_code', how='left')

# Now, you can create the attractions_per_capita column
# Replace 0 population with NaN to avoid division by zero
final_combined_data['population'].replace(0, np.nan, inplace=True)
final_combined_data['attractions_per_capita'] = final_combined_data['total_tourist_attractions'] / final_combined_data['population']

# Replace infinite values with NaN and then fill those NaNs
final_combined_data.replace([np.inf, -np.inf], np.nan, inplace=True)
final_combined_data['attractions_per_capita'].fillna(0, inplace=True)


### Feature Engineering

In [ ]:
# Example: Create a feature for the density of tourist attractions per capita
final_combined_data['attractions_per_capita'] = final_combined_data['total_tourist_attractions'] / final_combined_data['population']

# Replace infinite values with NaN and then handle those NaNs
final_combined_data.replace([np.inf, -np.inf], np.nan, inplace=True)
final_combined_data['attractions_per_capita'].fillna(0, inplace=True)


In [ ]:
final_combined_data

,poi_establishment,name_establishment,latitude_establishment,longitude_establishment,municipality_code,importance_establishment,establishment_name,poi_site,name_site,latitude_site,...,nb_tot_housing,secondary_home_rate,principal_home_rate,vacants_housing_rate,country_code_x,year_y,population,country_code_y,total_tourist_attractions,attractions_per_capita
0,hotel,Club Med (Hôtel),45.678531,-1.163635,17225,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,467.0,0.383298,0.554604,0.062099,1.0,1968.0,836.0,1.0,1.0,0.001196
1,hotel,Club Med (Hôtel),45.678531,-1.163635,17225,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,467.0,0.383298,0.554604,0.062099,1.0,1975.0,898.0,1.0,1.0,0.001114
2,hotel,Club Med (Hôtel),45.678531,-1.163635,17225,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,467.0,0.383298,0.554604,0.062099,1.0,1982.0,1005.0,1.0,1.0,0.000995
3,hotel,Club Med (Hôtel),45.678531,-1.163635,17225,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,467.0,0.383298,0.554604,0.062099,1.0,1990.0,1205.0,1.0,1.0,0.000830
4,hotel,Club Med (Hôtel),45.678531,-1.163635,17225,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,467.0,0.383298,0.554604,0.062099,1.0,1999.0,1452.0,1.0,1.0,0.000689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5331177,NaN,NaN,NaN,NaN,97801,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015.0,35684.0,1.0,NaN,0.000000
5331178,NaN,NaN,NaN,NaN,97801,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016.0,35746.0,1.0,NaN,0.000000
5331179,NaN,NaN,NaN,NaN,97801,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2017.0,35334.0,1.0,NaN,0.000000
5331180,NaN,NaN,NaN,NaN,97801,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2018.0,34065.0,1.0,NaN,0.000000


###Integrating Real Estate and Salary Data

Optimize Data Types

Merge on Index (If Applicable)

In [ ]:
# Set the 'municipality_code' as index on both the main dataframe and the ones to be merged
final_combined_data.set_index('municipality_code', inplace=True)
real_estate_info_by_municipality.set_index('municipality_code', inplace=True)
average_salary_by_municipality.set_index('municipality_code', inplace=True)

# Perform the merge using index matching, which is generally more efficient
final_combined_data = final_combined_data.join(real_estate_info_by_municipality, on='municipality_code', how='left', rsuffix='_realestate')
inal_combined_data = final_combined_data.join(average_salary_by_municipality, on='municipality_code', how='left', rsuffix='_salary')

# Reset index if necessary
final_combined_data.reset_index(inplace=True)


In [ ]:
# 'final_combined_data_dask' is my DataFrame
print(final_combined_data.columns)  # List all column names
print(final_combined_data)    # Show the first few rows of the DataFrame


Index(['municipality_code', 'poi_establishment', 'name_establishment',
       'latitude_establishment', 'longitude_establishment',
       'importance_establishment', 'establishment_name', 'poi_site',
       'name_site', 'latitude_site', 'longitude_site', 'importance_site',
       'site_name', 'year_x', 'nb_principal_home', 'nb_second_home',
       'nb_vacants_housing', 'nb_tot_housing', 'secondary_home_rate',
       'principal_home_rate', 'vacants_housing_rate', 'country_code_x',
       'year_y', 'population', 'country_code_y', 'total_tourist_attractions',
       'attractions_per_capita', 'intensite_tension_immo',
       'rental_max_apartment', 'rental_min_apartment', 'rental_med_house',
       'rental_max_house', 'rental_min_house', 'rental_med_all',
       'rental_max_all', 'rental_min_all'],
      dtype='object')
        municipality_code poi_establishment name_establishment  \
0                   17225             hotel   Club Med (Hôtel)   
1                   17225             ho

In [ ]:
final_combined_data

,municipality_code,poi_establishment,name_establishment,latitude_establishment,longitude_establishment,importance_establishment,establishment_name,poi_site,name_site,latitude_site,...,attractions_per_capita,intensite_tension_immo,rental_max_apartment,rental_min_apartment,rental_med_house,rental_max_house,rental_min_house,rental_med_all,rental_max_all,rental_min_all
0,17225,hotel,Club Med (Hôtel),45.678531,-1.163635,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,0.001196,8.0,17.88,10.65,11.52,16.05,8.56,11.96,16.57,9.16
1,17225,hotel,Club Med (Hôtel),45.678531,-1.163635,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,0.001114,8.0,17.88,10.65,11.52,16.05,8.56,11.96,16.57,9.16
2,17225,hotel,Club Med (Hôtel),45.678531,-1.163635,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,0.000995,8.0,17.88,10.65,11.52,16.05,8.56,11.96,16.57,9.16
3,17225,hotel,Club Med (Hôtel),45.678531,-1.163635,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,0.000830,8.0,17.88,10.65,11.52,16.05,8.56,11.96,16.57,9.16
4,17225,hotel,Club Med (Hôtel),45.678531,-1.163635,0.078556,Club Med,zoo,Zoo de la Palmyre,45.688217,...,0.000689,8.0,17.88,10.65,11.52,16.05,8.56,11.96,16.57,9.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5331177,97801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5331178,97801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5331179,97801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5331180,97801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Linear Regression Approach

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

### Applying Target Encoding ###

Target encoding is a powerful technique to handle categorical variables, especially when you have many unique categories. It replaces a categorical value with a number that represents the mean of the target variable for that category. This approach can be more meaningful than label encoding because it embeds information about the target within the feature, but it must be used carefully to avoid data leakage and overfitting.

In [ ]:
!pip install category-encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
import category_encoders as ce

# Define the columns you want to target encode
categorical_columns = ['poi_establishment', 'name_establishment', 'establishment_name', 'poi_site', 'name_site', 'site_name']

# Initialize and fit the target encoder
encoder = ce.TargetEncoder(cols=categorical_columns)

# Fit the encoder on the training data only to avoid leakage
X = final_combined_data.drop(['municipality_code', 'rental_med_all'], axis=1)
y = final_combined_data['rental_med_all'].fillna(final_combined_data['rental_med_all'].mean())

# Split the data first to avoid leakage
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply the encoder
X_train_encoded = encoder.fit_transform(X_train, y_train)
X_test_encoded = encoder.transform(X_test)


Handle Missing Values

In [ ]:
# Fill any remaining NaNs
X_train_encoded = X_train_encoded.fillna(X_train_encoded.mean())
X_test_encoded = X_test_encoded.fillna(X_test_encoded.mean())


Standardize Features

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


Train the Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize and train the Linear Regression model
linear_reg = LinearRegression()
linear_reg.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = linear_reg.predict(X_test_scaled)

# Calculate and print the Mean Squared Error and R-squared value
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r_squared = linear_reg.score(X_test_scaled, y_test)
print(f'R-squared: {r_squared}')


 Data Preprocessing

In [ ]:

X = final_combined_data.drop(['municipality_code', 'rental_med_all'], axis=1)  # Include all other feature columns
y = final_combined_data['rental_med_all']


# Remove non-numeric (categorical) columns
X = X.select_dtypes(exclude=['object'])


Scale Numeric Features

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Data Split

In [ ]:
# Starting again from the original DataFrame to ensure alignment
final_combined_data = final_combined_data.dropna(subset=['rental_med_all'])  # Drop rows where the target variable is NaN
X = final_combined_data.drop(['municipality_code', 'rental_med_all'], axis=1)  # Exclude non-feature columns
y = final_combined_data['rental_med_all']

# do the train-test split, ensuring that y doesn't contain NaNs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Fill NaN Values with Imputation

Drop NaN values from y_train and the corresponding observations in X_train

In [ ]:
# Drop NaN values from y_train and the corresponding observations in X_train
# You need to align the indexes of X_train and y_train after dropping NaNs from y_train
#y_train = y_train.dropna()
#X_train_imputed = X_train_imputed[y_train.index, :]  # Use the index of y_train to filter X_train_imputed


Initialize and Train the Linear Regression Model

In [ ]:
# Initialize the Linear Regression model
#linear_reg = LinearRegression()

# Fit the model to the training data (using the imputed training set and the corresponding y_train)
#linear_reg.fit(X_train_imputed, y_train)

# Then make predictions on the test set
#y_pred = linear_reg.predict(X_test_imputed)  # Use the imputed test set to make predictions

# Finally, evaluate the model using the test target variable (y_test)
#mse = mean_squared_error(y_test, y_pred)
#print(f'Mean Squared Error: {mse}')

#r_squared = linear_reg.score(X_test_imputed, y_test)
#print(f'R-squared: {r_squared}')


This is an exploratory cell

In [ ]:
# Identify non-numeric columns
#non_numeric_columns = X.select_dtypes(include=['object']).columns
#print(non_numeric_columns)


In [ ]:
#X = X.select_dtypes(exclude=['object'])

In [ ]:
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)


In [ ]:
# Split the data
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
#linear_reg = LinearRegression()
#linear_reg.fit(X_train, y_train)

# Make predictions and evaluate
#y_pred = linear_reg.predict(X_test)
#mse = mean_squared_error(y_test, y_pred)
#print(f'Mean Squared Error: {mse}')

#r_squared = linear_reg.score(X_test, y_test)
#print(f'R-squared: {r_squared}')
